# Lesson 1: Building blocks

# Language Models

1. Text LLMs: string -> string output
2. Chat model: list of messages -> single message output

In [7]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [8]:
import { ChatOpenAI } from "langchain/chat_models/openai";
import { HumanMessage } from "langchain/schema";

const model = new ChatOpenAI({
    modelName: "gpt-3.5-turbo-1106"
});

await model.invoke([
    new HumanMessage("Tell me a joke")
]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Why don't skeletons fight each other?\nThey don't have the guts!",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Why don't skeletons fight each other?\nThey don't have the guts!",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

# Prompt Template

In [ ]:
import { ChatPromptTemplate } from "langchain/prompts";

const prompt = ChatPromptTemplate.fromTemplate(
    `What are three good names for a company that makes {product}?`
);

In [9]:
await prompt.format({
    product: "colorful socks"
});

"Human: What are three good names ofr a company that makes colorful socks?"

In [11]:
await prompt.formatMessages({
    product: "colorful socks"
});

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names ofr a company that makes colorful socks?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names ofr a company that makes colorful socks?",
    name: undefined,
    additional_kwargs: {}
  }
]

In [12]:
import { 
  SystemMessagePromptTemplate, 
  HumanMessagePromptTemplate 
} from "langchain/prompts";

const promptFromMessages = ChatPromptTemplate.fromMessages([
  SystemMessagePromptTemplate.fromTemplate(
    "You are an expert at picking company names."
  ),
  HumanMessagePromptTemplate.fromTemplate(
    "What are three good names for a company that makes {product}?"
  )
]);

await promptFromMessages.formatMessages({
    product: "shiny objects"
})

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

In [13]:
const promptFromMessages = ChatPromptTemplate.fromMessages([
      ["system", "You are an expert at picking company names."],
      ["human", "What are three good names for a company that makes {product}?"]
])

await promptFromMessages.formatMessages({
    product: "shiny objects"
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

# LangChain Expression Language (LCEL)

In [14]:
const chain = prompt.pipe(model);

In [15]:
await chain.invoke({
    product: "colorful socks"
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "1. Rainbow Hues Socks Co.\n2. ChromaSock\n3. Vivid Threads Sock Company",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "1. Rainbow Hues Socks Co.\n2. ChromaSock\n3. Vivid Threads Sock Company",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

# Output Parser

In [16]:
import { StringOutputParser } from "langchain/schema/output_parser";

const outputParser = new StringOutputParser();

const nameGenerationChain = prompt.pipe(model).pipe(outputParser);

In [17]:
await nameGenerationChain.invoke({
  product: "fancy cookies",
});

"1. Decadent Delights\n2. Gourmet Gildings\n3. Luxe Bites"

https://smith.langchain.com/public/9fd7689f-23c6-4ea1-8a76-fea1b5f8d5db/r

# Streaming

In [19]:
const stream = await nameGenerationChain.stream({
    product: "really cool robots"
});

for await (const chunk of stream) {
    console.log(chunk);
}


1
.
 Future
Tech
 Robotics


2
.
 Rob
o
Gen
ius


3
.
 Mech
Master
 Inc
.



# Batch

In [20]:
const input = [
    {
        product: "large calculators"
    },
    {
        product: "alpaca wool sweaters"
    }
];

await nameGenerationChain.batch(input);

[
  "1. BigCalc Co.\n2. JumboMath Inc.\n3. HighCalcs Corp.",
  "1. AlpacaCozy\n2. AndeanThreads\n3. CosyCamelids"
]